In [1]:
import numpy as np
import keras

Using TensorFlow backend.


In [2]:
import gym
env = gym.make('Go9x9-v0')

[2017-10-03 13:57:50,834] Making new env: Go9x9-v0


In [3]:
import paths
from nn import policy9x9

In [4]:
def opponent(position1st, position2nd):
    """Return the opponent move with the position they saw
    
    Go9x9 gym environment makes whites moves automatically, so 
    it is not as easy to capture their state and move for training
    as it is for the black moves.
    This method calculates the position the white player saw and the 
    move they took in that position.
    
    :param positions___: np.array    3x9x9 arrays
    :return: (np.array, int)         (3x9x9 observation array, move integer)
    """
    mask = (position1st - position2nd)[1,:,:]
    if np.all(mask == 0):
        opp_move = 81
    else:
        opp_move = np.argmin(mask.reshape(81))
    
    mask = np.concatenate([np.zeros((1,9,9)), mask.reshape(1,9,9), -mask.reshape(1,9,9)],)
    opp_position = position2nd + mask

    return opp_position, opp_move

In [5]:
opponent(*observations[-2:])

NameError: name 'observations' is not defined

In [ ]:
observations[-1]

In [6]:
net = policy9x9.PolicyNet()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3, 9, 9)           0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 5, 11, 9)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 3, 9, 32)          2624      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 5, 11, 32)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 9, 32)          9248      
_________________________________________________________________
flatten_1 (Flatten)          (None, 864)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              885760    
__________

In [13]:
moves_made_per_game = []
total_reward = 0

for _ in range(100):
    observations = []
    moves = []
    
    opp_observations = []
    opp_moves = []
    
    obser = env.reset()    
    done = False
    
    while not done:
        observations.append(obser)
        move = net.move(position=obser)
        obser, reward, done, info = env.step(move)
        
        moves.append(move)
        
        opp_obser, opp_move = opponent(observations[-1], obser)
        opp_observations.append(opp_obser)
        opp_moves.append(opp_move)
    moves_made_per_game.append(len(moves))
    total_reward += reward
    
    moves = keras.utils.to_categorical(np.array(moves), num_classes=83)
    actionrewards = moves * reward    # uses the final reward as the reward for all moves
    observations = np.array(observations)
    
    net.train(observations=observations, actionrewards=actionrewards, verbose=0)
    
    opp_moves = keras.utils.to_categorical(np.array(opp_moves), num_classes=83)
    opp_actionrewards = opp_moves * (-reward)
    opp_observations = np.array(opp_observations)
    
    net.train(observations=opp_observations, actionrewards=opp_actionrewards, verbose=0)

np.median(moves_made_per_game), total_reward

(4.0, -93.0)

In [10]:
np.median(moves_made_per_game), total_reward

(4.0, -89.0)

In [12]:
np.median(moves_made_per_game), total_reward

(4.0, -91.0)